In [1]:
import gym
import ptan
import argparse
import torch
import torch.optim as optim
from tensorboardX import SummaryWriter
from lib import dqn_model, common

In [2]:
if __name__ == "__main__":
    #input hyperparameters, check CUDA available, create environment,then we use PTAN DQN wrapper to wrap up the environment
    params = common.HYPERPARAMS['pong']
    parser = argparse.ArgumentParser()
    parser.add_argument("--cuda", default=True, action="store_true", help="Enable cuda")
    args, unknown = parser.parse_known_args()
    device = torch.device("cuda" if args.cuda else "cpu")
    
    env = gym.make(params['env_name'])
    env = ptan.common.wrappers.wrap_dqn(env)
    
    #we make a writer for the environment and action dimension
    writer = SummaryWriter(comment="-" + params['run_name'] + "-basic")
    net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
    #the wrapper below can create a copy of DQN network, which is target network, and constantly synchronize with online
    #network
    tgt_net = ptan.agent.TargetNet(net)
    
    #we create agent to change observation to action value, we also need action selector to choose the action we use
    #We use epsilon greedy method as action selector here
    selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=params['epsilon_start'])
    epsilon_tracker = common.EpsilonTracker(selector, params)
    agent = ptan.agent.DQNAgent(net, selector, device=device)
    
    #experience source is from one step ExperienceSourceFirstLast and replay buffer, it will store fixed step transitions
    exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=params['gamma'], steps_count=1)
    buffer = ptan.experience.ExperienceReplayBuffer(exp_source, buffer_size=params['replay_size'])
    
    #create optimizer and frame counter
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    frame_idx = 0
    
    #reward tracker will report mean reward when episode end, and increase frame counter by 1, also getting a transition
    #from frame buffer.
    #buffer.populate(1) will activate following actions:
    #ExperienceReplayBuffer will request for next transition from experience source.
    #Experience source will send the observation to agent to get the action
    #Action selector which use epsilon greedy method will choose an action based on greedy or random
    #Action will be return to experience source and input to the environment to get reward and next observation, 
    # current observation, action, reward, next observation will be stored into replay buffer
    #transfer information will be stored in replay buffer, and oldest observation will be dropped
    with common.RewardTracker(writer, params['stop_reward']) as reward_tracker:
        while True:
            frame_idx += 1
            buffer.populate(1)
            epsilon_tracker.frame(frame_idx)
            
            #check undiscounted reward list after finishing an episode, and send to reward tracker to record the data
            #Maybe it just play one step or didn't have finished episode, if it returns true, it means the mean reward
            #reached the reward boundary and we can break and stop training
            new_rewards = exp_source.pop_total_rewards()
            if new_rewards:
                if reward_tracker.reward(new_rewards[0], frame_idx, selector.epsilon):
                    break
            
            #we check buffer has cached enough data to start training or not. If not, we wait for more data.
            if len(buffer) < params['replay_initial']:
                continue
            
            #here we use Stochastic Gradient Descent(SGD) to calculate loss, zero the gradient,batch from the replay buffer
            optimizer.zero_grad()
            batch = buffer.sample(params['batch_size'])
            loss_v = common.calc_loss_dqn(batch, net, tgt_net.target_model, gamma=params['gamma'], device=device)
            loss_v.backward()
            optimizer.step()
            
            #synchronize the target network with the online network constantly
            if frame_idx % params['target_net_sync'] == 0:
                tgt_net.sync()

879: done 1 games, mean reward -21.000, speed 195.16 f/s, eps 0.99
1639: done 2 games, mean reward -21.000, speed 225.75 f/s, eps 0.98
2617: done 3 games, mean reward -20.667, speed 214.22 f/s, eps 0.97
3528: done 4 games, mean reward -20.500, speed 172.31 f/s, eps 0.96
4429: done 5 games, mean reward -20.400, speed 146.31 f/s, eps 0.96
5268: done 6 games, mean reward -20.500, speed 132.00 f/s, eps 0.95
6181: done 7 games, mean reward -20.571, speed 156.99 f/s, eps 0.94
7226: done 8 games, mean reward -20.500, speed 159.40 f/s, eps 0.93
8057: done 9 games, mean reward -20.444, speed 159.57 f/s, eps 0.92
9173: done 10 games, mean reward -20.300, speed 158.30 f/s, eps 0.91
9931: done 11 games, mean reward -20.364, speed 161.55 f/s, eps 0.90
10892: done 12 games, mean reward -20.333, speed 58.79 f/s, eps 0.89
11711: done 13 games, mean reward -20.385, speed 55.28 f/s, eps 0.88
12701: done 14 games, mean reward -20.286, speed 54.79 f/s, eps 0.87
13591: done 15 games, mean reward -20.333, s

201248: done 120 games, mean reward -17.670, speed 57.54 f/s, eps 0.02
203865: done 121 games, mean reward -17.620, speed 58.20 f/s, eps 0.02
207024: done 122 games, mean reward -17.560, speed 58.09 f/s, eps 0.02
210535: done 123 games, mean reward -17.430, speed 57.84 f/s, eps 0.02
214118: done 124 games, mean reward -17.330, speed 58.06 f/s, eps 0.02
218117: done 125 games, mean reward -17.170, speed 58.16 f/s, eps 0.02
222786: done 126 games, mean reward -16.960, speed 57.91 f/s, eps 0.02
226388: done 127 games, mean reward -16.900, speed 57.66 f/s, eps 0.02
229692: done 128 games, mean reward -16.760, speed 57.91 f/s, eps 0.02
233257: done 129 games, mean reward -16.680, speed 57.78 f/s, eps 0.02
236592: done 130 games, mean reward -16.550, speed 58.13 f/s, eps 0.02
240029: done 131 games, mean reward -16.350, speed 57.69 f/s, eps 0.02
243533: done 132 games, mean reward -16.170, speed 58.21 f/s, eps 0.02
246744: done 133 games, mean reward -15.940, speed 57.62 f/s, eps 0.02
250098

469266: done 238 games, mean reward 16.930, speed 52.36 f/s, eps 0.02
470964: done 239 games, mean reward 17.040, speed 52.90 f/s, eps 0.02
472691: done 240 games, mean reward 17.220, speed 52.99 f/s, eps 0.02
474586: done 241 games, mean reward 17.260, speed 53.62 f/s, eps 0.02
476649: done 242 games, mean reward 17.290, speed 53.70 f/s, eps 0.02
478406: done 243 games, mean reward 17.470, speed 48.51 f/s, eps 0.02
480246: done 244 games, mean reward 17.510, speed 47.22 f/s, eps 0.02
481961: done 245 games, mean reward 17.620, speed 54.12 f/s, eps 0.02
483993: done 246 games, mean reward 17.660, speed 54.51 f/s, eps 0.02
485774: done 247 games, mean reward 17.700, speed 50.78 f/s, eps 0.02
487475: done 248 games, mean reward 17.810, speed 52.81 f/s, eps 0.02
489734: done 249 games, mean reward 17.750, speed 54.25 f/s, eps 0.02
491525: done 250 games, mean reward 17.780, speed 52.21 f/s, eps 0.02
493294: done 251 games, mean reward 17.860, speed 53.48 f/s, eps 0.02
494987: done 252 gam